In [4]:
import numpy as np
import pandas as pd
from sklearn.ensemble import *
import xgboost as xgb
import operator

!pip install ta
import ta

In [3]:
df = pd.read_csv('data.csv')

In [6]:
df.shape

(52974, 8)


In [6]:
df = df.dropna()

In [7]:
df.shape()

(52974, 8)


In [7]:
df['Close_target'] = df['Close'].shift(-1)
df = df.dropna()

In [7]:
df.loc[df.Close + (df.Close * settings.PERCENT_UP) < df.Close_target, "Target"] = 1 # 'UP'
df.loc[df.Close - (df.Close * settings.PERCENT_DOWN) > df.Close_target, "Target"] = 2 # 'DOWN'

In [8]:
print(df.shape)
print(len(df[df.Target == 1]), len(df[df.Target == 2]))

(52974, 8)
4867 4488


In [8]:
df['Date'] = df['Timestamp'].apply(utils.timestamptodate)
df['Date'] = pd.to_datetime(df['Date'])

df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Week'] = df['Date'].dt.weekofyear
df['Weekday'] = df['Date'].dt.weekday
df['Day'] = df['Date'].dt.day
df['Hour'] = df['Date'].dt.hour

In [11]:
df.shape

In [12]:
df = ta.add_all_ta_features(df, "Open", "High", "Low", "Close", "Volume_BTC", fillna=True)

In [13]:
train, test = ta.split(df)

In [14]:
excl = ['Close_target', 'Target', 'Date', 'Timestamp']
cols = [c for c in df.columns if c not in excl]

In [15]:
y_train = train['Target']
y_mean = np.mean(y_train)
xgb_params = {
    'n_trees': 800,
    'eta': 0.0045,
    'max_depth': 20,
    'subsample': 1,
    'colsample_bytree': 0.95,
    'colsample_bylevel': 0.95,
    'objective': 'multi:softmax',
    'num_class' : 3,
    'eval_metric': 'rmse',
    'base_score': 0,
    'silent': 1
}

dtrain = xgb.DMatrix(train[cols], y_train)
dtest = xgb.DMatrix(test[cols])

cv_result = xgb.cv(xgb_params, dtrain)

# train
model = xgb.train(xgb_params, dtrain, num_boost_round=705)

# predict
y_pred = model.predict(dtest)
y_true = test['Target']

ta.metrics(y_true, y_pred)

Accuracy: 0.77293791829479384
Coefficient Kappa: 0.22349834397494738
Classification Report:
             precision    recall

       KEEP       0.83      0.97
         UP       0.34      0.11
       DOWN       0.35      0.08


Confussion Matrix:
[[10609   192    90]
 [  953   164    81]
 [  939   122    94]]
